# Kratos example

In [ ]:
from probabilistic_library import ReliabilityProject, ReliabilityMethod, DistributionType, CombineProject, CombineType, CombinerMethod, CompareType
from probabilistic_library import UncertaintyProject, UncertaintyMethod, Stochast, StartMethod

### Model

In [ ]:
from run_kratos_quay_wall import zfuncwrapper_linear

### Variables

In [ ]:
def define_project_variables(project, sheetpile_thickness):
    use_correlations = False
    project.variables["geo_young_modulus"].is_array = True
    if use_correlations:
        project.variables["geo_young_modulus"].array_size = 2
        maxNrClays = 0
    else:
        project.variables["geo_young_modulus"].array_size = 7
        maxNrClays = 3

    for i in range(project.variables["geo_young_modulus"].array_size):
        young_modulus = Stochast()
        if i <= maxNrClays: # Clays with low-medium plasticity, medium
            young_modulus.distribution = DistributionType.uniform
            young_modulus.minimum = 5E6
            young_modulus.maximum = 8E6
        else: # Sand, uniform, medium
            young_modulus.distribution = DistributionType.deterministic
            young_modulus.mean = 40E6
        project.variables["geo_young_modulus"].array_variables.append(young_modulus)

    project.variables["interface_stiffness"].distribution = DistributionType.deterministic
    project.variables["interface_stiffness"].is_array = True
    project.variables["interface_stiffness"].array_size = 6
    project.variables["interface_stiffness"].mean = 1E11 # value checked

    project.variables["sheetpile_young_modulus"].distribution = DistributionType.deterministic
    project.variables["sheetpile_young_modulus"].is_array = True
    project.variables["sheetpile_young_modulus"].array_size = 1
    project.variables["sheetpile_young_modulus"].mean = 200E9 # value checked

    project.variables["sheetpile_poisson_ratio"].distribution = DistributionType.deterministic
    project.variables["sheetpile_poisson_ratio"].is_array = True
    project.variables["sheetpile_poisson_ratio"].array_size = 1
    project.variables["sheetpile_poisson_ratio"].mean = 0.0 # value checked

    project.variables["sheetpile_thickness"].distribution = DistributionType.deterministic
    project.variables["sheetpile_thickness"].is_array = True
    project.variables["sheetpile_thickness"].array_size = 1
    project.variables["sheetpile_thickness"].mean = sheetpile_thickness

    return project

### Uncertainty analysis

In [ ]:
project = UncertaintyProject()
wrapper = zfuncwrapper_linear(output_stage_number = 3, clean_up = True)
project.model = wrapper.total_displacement

sheetpile_thickness = 0.015
project = define_project_variables(project, sheetpile_thickness)

project.settings.uncertainty_method = UncertaintyMethod.crude_monte_carlo
project.settings.variation_coefficient = 0.02
project.settings.minimum_samples = 50
project.settings.maximum_samples = 100
project.settings.save_convergence = True
project.settings.save_realizations = True

project.run()

project.stochast.plot()

### Reliability analysis, prior

Expected sheet pile thickness in year 2075 is 0.015 m

In [ ]:
project = ReliabilityProject()
wrapper = zfuncwrapper_linear(output_stage_number = 3, clean_up = True)
project.model = wrapper.total_displacement

sheetpile_thickness = 0.015
project = define_project_variables(project, sheetpile_thickness)

project.limit_state_function.parameter = project.model.output_parameters[0]
project.limit_state_function.compare_type = CompareType.less_than
project.limit_state_function.critical_value = -0.18

project.settings.reliability_method = ReliabilityMethod.form
project.settings.epsilon_beta = 0.02
project.settings.start_method = StartMethod.sensitivity_search
project.settings.maximum_iterations = 50
project.settings.save_convergence = True
project.settings.save_realizations = True

project.run()

project.design_point.print()
project.design_point.plot_alphas()
project.design_point.plot_convergence()
project.design_point.plot_realizations()

prior = project.design_point

### Survived event

Sheet pile thickness in 2025 is 0.025 and the total displacement is less than the critical value of -0.18 m

In [ ]:
sheetpile_thickness = 0.025
define_project_variables(project, sheetpile_thickness)

project.limit_state_function.parameter = project.model.output_parameters[0]
project.limit_state_function.compare_type = CompareType.greater_than
project.limit_state_function.critical_value = -0.18

project.settings.reliability_method = ReliabilityMethod.form
project.settings.epsilon_beta = 0.02
project.settings.start_method = StartMethod.sensitivity_search
project.settings.maximum_iterations = 50
project.settings.save_convergence = True
project.settings.save_realizations = True

project.run()

project.design_point.print()
project.design_point.plot_alphas()
project.design_point.plot_convergence()
project.design_point.plot_realizations()

survived_event = project.design_point

### Combined probability

In [ ]:
combine_project = CombineProject()

combine_project.design_points.append(prior)
combine_project.design_points.append(survived_event)

combine_project.settings.combine_type = CombineType.parallel
combine_project.settings.combiner_method = CombinerMethod.hohenbichler_form

combine_project.run()

combine_project.design_point.print()

comb = combine_project.design_point

### Reliability, posterior

In [ ]:
prob_posterior = comb.probability_failure/survived_event.probability_failure

print(prob_posterior)